# reviewing theorem 3

- $C$: number of cases
- $\epsilon$: fraction of cases
- $k$: the size of a set of *any* individuals in the population such that there are at least 2 individuals that differ on $\epsilon C$ cases
    - smaller $k$ makes this harder to achieve
    - larger $\epsilon$ makes this harder to achieve

empirical question:
- what is the smallest $k$, and largest $\epsilon$, that satisfies these constraints?

## approach

- load pop error matrix, B
- set $\epsilon$ to some value

- make NxN adjacency matrix where i,j indicates whether $n_i$ and $n_j$ differ on less than $\epsilon$C cases
- calculate the maximum clique of this matrix, whose size equals $k$ 

# todo

- compare behavioral diversity to values of k, epsilon
    - calculate mean proportion of unique error vectors each gen
- calculate best epsilon per generation? 
    - look at runtime fraction with optimal epsilon?
    - what is the best runtime fraction using optimal epsilon, over generations?
- compare empirical # evals for these runs to new estimate/bound of running time 



In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
import matplotlib.pyplot as plt
def draw_graph(G):
    plt.figure()
    nx.draw(G)

# generate data
- can turn this off if loading from saved state.

In [ ]:
from scipy.spatial.distance import hamming
import itertools as it
from tqdm.notebook import  tqdm
from glob import glob
import warnings
import signal
import time
import timeit 
    
class TimeOutException(Exception):
   pass

def alarm_handler(signum, frame):
#     print("ALARM signal received")
    raise TimeOutException()
    
MAXTIME = 10
def average_different(x, y):
    """count the number of elements where x and y differ, divide by length"""
    assert x.shape == y.shape
    return (x != y).sum()/len(x)
# for g in tqdm(df.generation.unique()):
def save_runtime_stats(datadir, boolean):
    comparison = hamming if boolean else average_different
    filelist = list(glob(datadir+f'/*errors*.csv'))
    print(f'\n{datadir}, trials: {len(filelist)}, errors: {"boolean" if boolean else "integer"}\n')
    problem = datadir.split('lex-theory-')[-1].replace('-',' ').title()
    for t,f in enumerate(tqdm(filelist, problem+' trials')):
        frames = []
        df = pd.read_csv(f)
        for g in tqdm(df.generation.unique(), 
                      '--- '+ problem + f' trial {t}',
                      leave=False):
#         for g in df.generation.unique()[:50]:
#         for g in df.generation.unique():
            result = {}
            B = df.loc[df.generation==g,[c for c in df.columns if 'TC' in c]]
#             print('B shape:',B.shape)
            pop_size = B.shape[0]
            # remove duplicate arrays
            B = B.drop_duplicates()
            nunique = len(B)/pop_size
            B = B.values
            # calculate the determinant of the covariance matrix
#             det_cov = np.linalg.det(np.cov(B))
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                X = np.corrcoef(B)
                X = X[np.logical_not(np.isnan(X))]
                mean_cov = np.mean(np.abs(X))
#                 print('corrcoef:',X)
#                 print('mean_cov:',mean_cov)
            C = B.shape[1]
            N = B.shape[0]
            eps = 0.5
            # pairwise distance
            D = np.empty((N,N))
            for i,j in it.product(np.arange(len(B)), repeat=2):
                D[i,j] = comparison(B[i,:],B[j,:])
            # epsilon
            best_eps = None
            runtimemin = None
            subframes = []
#             if problem != 'Compare String Lengths':
#                 eps_range = np.linspace(0.05,0.6,12)  
#             else:
#                 np.linspace(0.05,0.45,9) 
            eps_range = np.linspace(0.05,0.6,12)  
            for eps in tqdm(eps_range,
                            '------ '+ problem + f' trial {t} gen {g}', 
                            leave=False): 
                A = D < eps
                # max clique
                G = nx.convert_matrix.from_numpy_matrix(A)
                # define timeout signal
                signal.signal(signal.SIGALRM, alarm_handler)
                signal.alarm(MAXTIME)
                start = time.process_time()
                try:
                    k = nx.algorithms.clique.graph_clique_number(G)
                except TimeOutException as e:
#                     print(e, 'problem:',problem, 'eps:',eps)
#                     k = float('NaN')
                    break
                signal.alarm(0)
                clique_time=time.process_time() - start
#                 if problem == 'Compare String Lengths':
#                     print('eps: ',eps, 'clique time: ', clique_time)
                    
                runtime = 4*N/eps + 2*k*C
                
                if best_eps == None:
                    best_eps = eps
                    runtimemin = runtime
                elif runtime < runtimemin:
                    best_eps = eps
                    runtimemin = runtime
                    
                subframes.append({
                    'problem':problem,
                    'g':g,
                    'pop_size':pop_size,
                    'N':N,
                    'k':k,
                    'k%':round(k/N*100,2),
                    'eps':eps,
                    'runtime bound':runtime,
                    'N*C': N*C,
                    'runtime fraction': (runtime)/(N*C),
                    'trial':t,
                    'nunique':nunique,
                    'mean_cov':mean_cov,
                    'clique time': clique_time
                })
            for i,sf in enumerate(subframes):
                subframes[i]['best_eps'] = best_eps
                subframes[i]['runtime min'] = runtimemin
                
            frames += subframes
    
        print(f'saving stats for {f}')
        res = pd.DataFrame.from_records(frames)
        
        res.to_csv(datadir+f'/runtime_stats-{t}.csv', index=False)

In [ ]:
rootdir = 'data/lex-theory-'
# popsize = '250'
# problem = 'mirror-image'
# problems = [
#             ('compare-string-lengths',True,),
#             ('count-odds',False),
#             ('double-letters',False),
#             ('last-index-of-zero',False),
#             ('mirror-image',True),
#             ('negative-to-zero',False)
# ]
problems = [
            ('x-word-lines', False),
            ('vector-average', False)
#             ('compare-string-lengths',True,),
#             ('count-odds',False),
#             ('double-letters',False),
#             ('last-index-of-zero',False),
#             ('mirror-image',True),
#             ('negative-to-zero',False)
]
# from multiprocessing import Pool
from pqdm.processes import pqdm

args =[(rootdir+name, boolean) for name,boolean in problems] 
pqdm(args, save_runtime_stats, n_jobs=len(args), argument_type='args')
# with Pool(processes=len(problems)) as pool:
#     pool.starmap(save_runtime_stats, args )
#     for name, boolean in problems.items():
#         datadir = rootdir+name
#         save_runtime_stats(datadir, boolean) 